In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
bbn_path = base_dir + "BBN/BBN/"

Mounted at /content/drive/


In [2]:
import json

with open(bbn_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [3]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  32739 elementi


In [10]:
print(examples_train[14496])

{'tokens': ['Officials', 'in', 'some', 'of', 'these', 'states', 'say', 'they', 'do', "n't", 'want', 'to', 'simply', 'accept', 'the', 'settlements', 'offered', 'by', 'Drexel', '.'], 'senid': 6, 'mentions': [{'start': 18, 'labels': ['/ORGANIZATION/CORPORATION', '/ORGANIZATION', '/ORGANIZATION/EDUCATIONAL'], 'end': 19}, {'start': 0, 'labels': ['/PERSON'], 'end': 1}], 'fileid': 'WSJ1296'}


In [3]:
examples_iter = iter(examples_train)

In [ ]:
import itertools
from tqdm import tqdm
fp = open(path_corpus_data + 'corpus_data_bbn.txt', "w+")  # Open a file
phrase = []
restricted_set = set()
for i in tqdm(examples_iter):
  all_list = []
  permutations = []
  if len(i['mentions'])>1:
    # creo tutte le permutazioni
    for j in i['mentions']:
      all_list.append(j['labels'])

    permutations = iter(list(itertools.product(*all_list)))
    del all_list

    # creo tutte le possibili frasi a partire dalle permutazioni
    for p in permutations:
      phrase = []
      phrase = i['tokens'].copy()
      for h in range(0, len(p)):
        phrase[i['mentions'][h]['start']] = p[h]
        restricted_set.add(p[h])
      fp.write(' '.join(phrase) + '\n')
      del phrase
    
  else:
    for k in i['mentions'][0]['labels']:
      phrase = []
      phrase = i['tokens'].copy()
      phrase[i['mentions'][0]['start']] = k
      restricted_set.add(k)
      fp.write(' '.join(phrase) + '\n')
      del phrase

fp.close()

14093it [00:02, 6066.21it/s]

In [ ]:
restricted_list = list(restricted_set)

In [ ]:
import pickle
with open(path_corpus_data + 'restricted_list_bbn.pkl', 'wb') as fp:
  pickle.dump(restricted_list, fp)